Zalecamy nie czytać notatników na githubie, ze względu na źle wyświetlające się wizualizacje i brak możliwości uruchamiania kodu. Polecamy otworzyć notatnik w google colab.

#**Olimpiada AI - kurs wprowadzający 2025 - Wykład 001E**

"Kontrola losowości oraz wizualizacja postępu"

## Wizualizacja postępu - biblioteka `tqdm`

Dziwna nazwa `tqdm` pochodzi od arabskiego słowa *taqaddum* (تقدّم), które oznacza "postęp".

I dokładnie to nam daje – wizualizację postępu.


In [1]:
#Importy bibliotek potrzebne do przykładów
import time
import random
import pandas as pd

### Jak używać?

Zamiast pisać `for i in range(100):`, owijamy `range(100)` w `tqdm()`.

In [2]:
from tqdm import tqdm

In [3]:
print("Symulacja liczenia w pętli ...")
for i in tqdm(range(100)):
    time.sleep(0.02) # Używamy funkcji "sleep", która po prostu czeka

Symulacja liczenia w pętli ...


100%|██████████| 100/100 [00:02<00:00, 47.91it/s]


 💡 **Pro-Tip:** W Jupyter Notebooku dzięki `tqdm.auto` pasek będzie graficzny.

In [4]:
# nadpisujemy inport
# tqdm.auto automatycznie wybiera wersję (konsola vs. notebook)
# W Jupyter/Colab da nam ładny, graficzny pasek (HTML)
from tqdm.auto import tqdm

In [5]:
print("Symulacja liczenia w pętli ...")
for i in tqdm(range(100)):
    time.sleep(0.02) # Używamy krótszego snu dla szybszego demo w notebooku

Symulacja liczenia w pętli ...


  0%|          | 0/100 [00:00<?, ?it/s]

### Dodawanie kontekstu - `desc`

Gdy uruchamiasz wiele pętli, same paski postępu nic Ci nie mówią. Musisz wiedzieć, *co* się dzieje. Służy do tego argument `desc` (description), który ustawia statyczny prefiks przed paskiem.

In [6]:
# Wyobraźmy sobie, że mamy listę plików do wczytania
files_to_process = ['data_batch_1.csv', 'data_batch_2.csv', 'data_batch_3.csv']

for f in tqdm(files_to_process, desc="Wczytywanie plików"):
    time.sleep(0.5) # Symulacja wczytywania i przetwarzania pliku

for i in tqdm(range(200), desc="Augmentacja (krok 1/2)"):
    time.sleep(0.005) # Symulacja augmentacji

for i in tqdm(range(100), desc="Augmentacja (krok 2/2)"):
    time.sleep(0.01) # Symulacja augmentacji

Wczytywanie plików:   0%|          | 0/3 [00:00<?, ?it/s]

Augmentacja (krok 1/2):   0%|          | 0/200 [00:00<?, ?it/s]

Augmentacja (krok 2/2):   0%|          | 0/100 [00:00<?, ?it/s]

### Dynamiczne Informacje - `set_postfix()`

To jest **najważniejszy trik** dla każdego, kto trenuje modele. `desc` jest statyczny, ale co, jeśli chcemy na bieżąco monitorować np. *loss* (funkcję straty) albo *accuracy* (dokładność)?

Używamy metody `.set_postfix()`. Aby to zrobić, musimy mieć dostęp do obiektu paska postępu, więc użyjemy składni `with`.

In [7]:
# Tworzymy "ręcznie" obiekt paska postępu, aby mieć do niego dostęp
# (używamy 'with', żeby pasek poprawnie się zamknął)
print("Rozpoczynamy trening...")

with tqdm(range(100), desc="Epoka 1/5") as pbar:
    current_loss = 1.0
    for i in pbar:
        # Symulacja kroku treningowego
        time.sleep(0.02)

        # Strata "maleje" (losowo, dla przykładu)
        current_loss -= random.uniform(0.001, 0.01)

        # Aktualizujemy informacje!
        # Możemy przekazać słownik, który tqdm ładnie sformatuje
        pbar.set_postfix(loss=f"{current_loss:.4f}", accuracy=f"{i/100.0:.2%}")

Rozpoczynamy trening...


Epoka 1/5:   0%|          | 0/100 [00:00<?, ?it/s]

### Zagnieżdżone pętle `tqdm`

Co się stanie, gdy wsadzimy `tqdm` w `tqdm`? W notebooku Jupyter jest *trochę* lepiej niż w czystej konsoli, ale wciąż potrafi zrobić bałagan (paski zostają po zakończeniu).


In [14]:
epochs = 3
batches = 200

# Pętla zewnętrzna (Epoki)
# Domyślnie position=0, leave=True
for epoch in tqdm(range(epochs), desc="Epoki"):

    # Pętla wewnętrzna (Batche)
    # leave=False jest kluczowe, aby pasek zniknął po zakończeniu epoki
    # Usunięcie position zapobiega przesuwaniu paska w dół
    for batch in tqdm(range(batches), desc=f"Epoch {epoch+1} Batches", leave=True):
        time.sleep(0.01) # Bardzo krótki sen

    # Symulacja walidacji na koniec epoki
    time.sleep(1.0)

Trening zagnieżdżony (poprawnie):


Epoki:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1 Batches:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch 2 Batches:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch 3 Batches:   0%|          | 0/200 [00:00<?, ?it/s]

## Kontrola Losowości - czyli jak zapewnić powtarzalność wyników?



Wyobraźcie sobie: trenujecie model, osiągacie 95% skuteczności. Super! Uruchamiacie skrypt jeszcze raz... i dostajecie 92%. A za trzecim razem 94%. Dlaczego?

Odpowiedź: Niekontrolowana losowość.

Nauka polega na powtarzalności. Jeśli nie potrafisz odtworzyć swojego wyniku, to tak, jakby go nie było. W AI "wrogami" powtarzalności są:

1.  Losowa inicjalizacja wag w sieciach neuronowych.
2.  Losowy podział na zbiór treningowy i testowy (`train_test_split`).
3.  Losowe operacje w algorytmach (np. tasowanie danych, dropout, augmentacja).


### Rozwiązanie: "Ziarno" (Seed)

Musimy "zamrozić" losowość. Robimy to, ustawiając tzw. **ziarno pseudolosowości (seed)**. To liczba (dowolna, ale musi być ta sama), która inicjuje generator liczb losowych. Jeśli ziarno jest takie samo, sekwencja "losowych" liczb też będzie taka sama.

In [10]:
import random
import numpy as np
import tensorflow as tf
import torch
import os

# Ustaw globalne ziarno
SEED = 42

def set_seed(seed=SEED):
    # Ustaw ziarno dla podstawowego modułu 'random'
    random.seed(seed)

    # Ustaw ziarno dla Numpy
    np.random.seed(seed)

    # Ustaw ziarno dla TensorFlow
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)

    # Ustaw ziarno dla PyTorch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # dla wielu GPU

    # Ustawienia dla determinizmu operacji CUDA (GPU)
    # Uwaga: to może spowolnić trening!
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Wywołaj funkcję ustawiającą ziarno
set_seed()

print(f"Wszystkie ziarna ustawione na {SEED}")

Wszystkie ziarna ustawione na 42


Pamiętajcie też, że można ustawiać `random_state=SEED` we wszystkich funkcjach scikit-learn lub PyTorch, które mają taki parametr:

```python
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

# Przykład
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

kmeans = KMeans(n_clusters=3, random_state=SEED)
```